In [1]:
from statsmodels.tsa.arima_model import ARIMA
import pandas as pd
from urllib.request import Request, urlopen
import json
import time
import requests
from datetime import datetime as dt
import os
import sys
from datetime import date, timedelta

In [2]:
def forecast_price(df):
    '''
    Requires df with ticker index
    '''
    fut_df = pd.DataFrame()
    forecast_df=pd.DataFrame()
    forecast = {}
    for i in range(len(df)):
        
        try:
            
            ticker = df.Ticker[i]
            ohlcv = get_crypto_daily_price(ticker)
            fut_df[ticker] = ohlcv['close']

            for i in progressbar(range(10), f"Query Success for {ticker}!, Preparing data for next call: ", 40):
                    time.sleep(0.1)

            print(f'Building a model for {ticker}')
            model = ARIMA(fut_df[ticker].dropna(), order=(4,1,4))
            results = model.fit()
            forecast[ticker] = results.forecast(steps=30)[0]
            
        except:
            print('error')
            pass
    

        
    return forecast

In [3]:
def get_crypto_daily_price(ticker):
    ticker = ticker.upper()
    api_key = os.getenv("CC_API")
    crypto_df = pd.DataFrame()
    url = f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={ticker}&tsym=USD&tryConversion=0&allData=true&api_key={api_key}"
    raw_data = read_json(url)
    df = pd.DataFrame(raw_data['Data']['Data'])
    df['time'] = pd.to_datetime(df['time'],unit='s')
    df.set_index(df['time'], inplace=True)
    df['close'] = df['close'].astype(float)
    df['var'] = df['close'].pct_change()
    return df.dropna()

In [4]:
def read_json(url):
    request = Request(url)
    response = urlopen(request)
    data = response.read()
    url2 = json.loads(data)
    return url2

In [5]:
def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

In [6]:
def daterange(current_date, end_date):
    current_date = date.today()
    end_date = (date.today()+timedelta(days=29))
    current_date.strftime("%Y-%m-%d")
    end_date.strftime("%Y-%m-%d")
    for n in range(int((end_date - current_date).days)+1):
        yield current_date + timedelta(n)


In [7]:
df = pd.read_csv('../../data/cleandata/success_df.csv')

In [ ]:
ARIMA_30day_forecast = forecast_price(df)

Query Success for ETH!, Preparing data for next call: [########################################] 10/10
Building a model for ETH


C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


Query Success for BNB!, Preparing data for next call: [########################################] 10/10
Building a model for BNB
error


C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


Query Success for EOS!, Preparing data for next call: [########################################] 10/10
Building a model for EOS


C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\Users\cscat\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


In [88]:
ARIMA_30day_forecast_df = pd.DataFrame(ARIMA_30day_forecast)

In [89]:
df_index = []
for dt in daterange(start_dt, end_dt):
    df_index.append(dt.strftime("%Y-%m-%d"))

In [90]:
ARIMA_30day_forecast_df['date']=df_index

In [91]:
ARIMA_30day_forecast_df.set_index('date', inplace=True)

In [92]:
ARIMA_30day_forecast_df.to_csv('../../data/cleandata/ARIMA_30day_forecast_df.csv')

In [107]:
ARIMA_30day_forecast_df['CVC']

date
2019-11-17    0.037297
2019-11-18    0.033572
2019-11-19    0.029695
2019-11-20    0.026634
2019-11-21    0.023999
2019-11-22    0.021492
2019-11-23    0.019123
2019-11-24    0.016882
2019-11-25    0.014729
2019-11-26    0.012644
2019-11-27    0.010622
2019-11-28    0.008655
2019-11-29    0.006740
2019-11-30    0.004872
2019-12-01    0.003049
2019-12-02    0.001268
2019-12-03   -0.000472
2019-12-04   -0.002172
2019-12-05   -0.003835
2019-12-06   -0.005461
2019-12-07   -0.007052
2019-12-08   -0.008609
2019-12-09   -0.010133
2019-12-10   -0.011625
2019-12-11   -0.013086
2019-12-12   -0.014518
2019-12-13   -0.015921
2019-12-14   -0.017296
2019-12-15   -0.018644
2019-12-16   -0.019967
Name: CVC, dtype: float64

In [15]:
x = get_crypto_daily_price('XTZ').iloc[0]
x

close                            0.7175
conversionSymbol                       
conversionType                   direct
high                               0.95
low                                0.61
open                             0.7894
time                2017-07-05 00:00:00
volumefrom                      23951.6
volumeto                        19393.7
var                          -0.0910818
Name: 2017-07-05 00:00:00, dtype: object